# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# Импорт необходимых библиотек

import pandas as pd
import numpy as np

In [2]:
# Чтение из данных и создание переменной scoring_data
scoring_data = pd.read_csv('/datasets/data.csv')

In [3]:
# Вывод основной информации
scoring_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# Вывод первых 15 строк для первичного осмотра даннных
scoring_data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
# Вывод описания таблицы
scoring_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [6]:
# Группировка по количеству детей
scoring_data.groupby('children').count()

,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
children,,,,,,,,,,,
-1,44,47,47,47,47,47,47,47,47,44,47
0,12710,14149,14149,14149,14149,14149,14149,14149,14149,12710,14149
1,4343,4818,4818,4818,4818,4818,4818,4818,4818,4343,4818
2,1851,2055,2055,2055,2055,2055,2055,2055,2055,1851,2055
3,294,330,330,330,330,330,330,330,330,294,330
4,34,41,41,41,41,41,41,41,41,34,41
5,8,9,9,9,9,9,9,9,9,8,9
20,67,76,76,76,76,76,76,76,76,67,76


`Можно сделать вывод о том что значения -1 и 20 являются ошибкам при вводе данных пользователем`

**Вывод**

Столбцы ***days_employed*** и ***total_income*** имеют пустые значения.

47 клиентов указали количество детей равной ***-1***, 76 клиентов указали ***20***

В столбце ***days_employed*** отрицательные значения, и некоторые значения слишком большие

В столбце ***education*** нет единого формата записи

101 клиент указали возраст ***dob_years*** равным ***нулю***

71 строк являются дубликатами

## Шаг 2. Предобработка данных

In [7]:
# Подведем к единому форматированию стобец образования
scoring_data['education'] = scoring_data['education'].apply(str.lower)

---

In [8]:
# Исправим ошибки в столбце с количеством детей.
# Вместо -1 поставим 1
# Вместо 20 поставим 2
scoring_data.loc[scoring_data.loc[:, 'children'] == -1, 'children'] = 1
scoring_data.loc[scoring_data.loc[:, 'children'] == 20, 'children'] = 2

### Обработка пропусков

In [9]:
# Количество явных пропусков
scoring_data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

---

In [10]:
# Проверим столбец days_employed чтобы найти проблему с отрицательными значениями
print(f'Клиенты с отрицательным стажем {(scoring_data["days_employed"] < 0).sum()}')
print(f'Клиенты с положительным стажем {(scoring_data["days_employed"] > 0).sum()}')
print(f'Клиенты с не заполненным стажем {scoring_data["days_employed"].isna().sum()}')

Клиенты с отрицательным стажем 15906
Клиенты с положительным стажем 3445
Клиенты с не заполненным стажем 2174


In [11]:
# Рассмотрим стаж в годах для каждой из групп
abs((scoring_data["days_employed"][scoring_data["days_employed"] < 0]) / 365).sort_values()
# Можно смело отрицательные значения переписать по модулю
scoring_data["days_employed"] = scoring_data["days_employed"].abs()

In [12]:
print(f'Клиенты с отрицательным стажем {(scoring_data["days_employed"] < 0).sum()}')
print(f'Клиенты с положительным стажем {(scoring_data["days_employed"] > 0).sum()}')
print(f'Клиенты с не заполненным стажем {scoring_data["days_employed"].isna().sum()}')

Клиенты с отрицательным стажем 0
Клиенты с положительным стажем 19351
Клиенты с не заполненным стажем 2174


Можно предположить что проблема с данными по стажу связана с тем что при расчете стажа для пенсионеров и безработных в таблицу занесены не дни а часы.

---

In [13]:
# Категории клиентов по типу работы среди пропусков
scoring_data.loc[scoring_data['days_employed'].isna() == True, ['income_type']].groupby('income_type').sum()

""
income_type
госслужащий
компаньон
пенсионер
предприниматель
сотрудник


In [14]:
# Заполним пропуски в столбце total_income средними по значениями по типу профессии
scoring_data['total_income'] = scoring_data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

---

In [15]:
# Заполним пропуски в столбце days_employed средним значением по возрасту клиента
scoring_data['days_employed'] = scoring_data.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean() * scoring_data['dob_years'] * 365))

In [16]:
# Количество явных пропусков после обработки
scoring_data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [17]:
#Решим последную проблему связанную с возрастом клиентов, подставив среднее значение по типу работы
scoring_data["dob_years"] = scoring_data.groupby("income_type")["dob_years"].transform(lambda x: x.replace(0, x.mean()))

In [18]:
scoring_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,2.152500e+04,21525.00000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,1.522834e+08,43.49822,0.817236,0.972544,0.080883,1.652253e+05
std,0.755528,1.078074e+09,12.22943,0.548138,1.420324,0.272661,9.804367e+04
min,0.000000,0.000000e+00,19.00000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1.024652e+03,34.00000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2.605748e+03,43.00000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,3.336411e+05,53.00000,1.000000,1.000000,0.000000,1.955499e+05
max,5.000000,9.725518e+09,75.00000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Все пропуски заполнены средними значениями по группам.

Столбец образования подведен к единому формату.

Обработаны ошибки с количеством детей.

Обработан столбец с возрастом.

### Замена типа данных

In [19]:
scoring_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


In [20]:
scoring_data["days_employed"] = scoring_data["days_employed"].astype('int')
scoring_data["dob_years"] = scoring_data["dob_years"].astype('int')
scoring_data["total_income"] = scoring_data["total_income"].astype('int')

In [21]:
scoring_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Произвел замену вещественных чисел на целочисленный в столбцах ***days_employed, dob_years, total_income***

astype использовался в силу того что после об

### Обработка дубликатов

In [22]:
# Количество дубликатов
scoring_data.duplicated().sum()

71

In [23]:
# Удаляем дубликаты
scoring_data = scoring_data.drop_duplicates().reset_index(drop=True)

**Вывод**

Очистил данные от дубликатов, причина скорее всего просто ошибка при записи данных

### Лемматизация

In [24]:
# Импорт библиотеки для лемматизации 
from pymystem3 import Mystem
m = Mystem()

In [25]:
# Создадим функцию для обработки списка леммы что бы избавится от про
def treatment_lemo_list(arr):
    result = list()
    for value in arr:
        if value == ' ' or value == '\n' or len(value) < 5:
            continue
        result.append(value)
    return result

In [26]:
# Рассмотрим цели кредитования и проведемм леммитизацию
lemo_list = []
for goal in scoring_data['purpose']:
    lemo_list.extend(treatment_lemo_list(m.lemmatize(goal)))

In [27]:
from collections import Counter
lemo_dict = Counter(lemo_list)

for goal_lemo, count in sorted(lemo_dict.items(), key=lambda x: x[1], reverse=True):
    print(f"{goal_lemo} - {count}")

недвижимость - 6351
покупка - 5897
жилье - 4460
автомобиль - 4306
образование - 4013
операция - 2604
свадьба - 2324
строительство - 1878
высокий - 1374
получение - 1314
коммерческий - 1311
жилой - 1230
сделка - 941
дополнительный - 906
заниматься - 904
проведение - 768
сыграть - 765
сдача - 651
семья - 638
собственный - 635
ремонт - 607
подержанный - 486
подержать - 478
приобретение - 461
профильный - 436


Теперь выберим топ - 10 слов, и создадим список для его дальнейшего редактирования для сздания категорий по целям.

In [28]:
# Сделаем срез по отсортированому словарю.
result_list = [value[0] for value in sorted(lemo_dict.items(), key=lambda x: x[1], reverse=True)[:10]]
[value[0] for value in sorted(lemo_dict.items(), key=lambda x: x[1], reverse=True)[:10]]

['недвижимость',
 'покупка',
 'жилье',
 'автомобиль',
 'образование',
 'операция',
 'свадьба',
 'строительство',
 'высокий',
 'получение']

Теперь из полученного списка нужно убрать те слова которые являются общими.
***покупка, операция, строительство, высокий, получение***

In [29]:
erase = ['покупка', 'операция', 'строительство', 'высокий', 'получение']
for value in erase:
    result_list.remove(value)

In [30]:
# Теперь мы получили список из основных целей взятия кредита.
result_list

['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']

In [31]:
purpose_tempory = pd.DataFrame(result_list, columns=['purpose_lem'])
purpose_tempory['purpose_id'] = [1,1,2,3,4]
purpose_tempory

,purpose_lem,purpose_id
0,недвижимость,1
1,жилье,1
2,автомобиль,2
3,образование,3
4,свадьба,4


**Вывод**

Вывел список с целями кредитования. Теперь необходимо создать столбец с категориями по целям покупки.

### Категоризация данных

На основе результатов лемматизации создадим список содержащий цель и уникальный id для этой цели.

Как видно по списку целей мы можем объединить недвижимость и жилье в одну категорию.

In [32]:
def purpose_categ(purpose):
    buf = m.lemmatize(purpose)
    for index in range(len(purpose_tempory)):
        if purpose_tempory['purpose_lem'][index] in buf:
            return purpose_tempory['purpose_id'][index]

In [33]:
scoring_data['purpose_id'] = scoring_data['purpose'].apply(purpose_categ)

In [34]:
scoring_data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4


---
Создадим функцию для создания категории клиентов по заработку
Разделим по следующим категориям
- бедность         (income < 25k)
- низкий средний   (25k < income < 50k)
- средний класс    (50к < income < 140k)
- высокий средний  (140k < income < 250k)
- богатый          (250k < income < 500k)
- Очень богатые    (500k < income)

In [35]:
def income_categ(income):
    if (income < 60000):
        return 'бедность [_60k]'
    elif (60000 < income < 120000):
        return 'низкий средний [60k_120K]'
    elif (120000 < income < 170000):
        return 'средний класс [120k_170K]'
    elif 170000 < income < 250000:
        return 'высокий средний [170k_250K]'
    elif (250000 < income < 500000):
        return 'богатый [250k_500K]'
    elif income > 500000:
        return 'Очень богатый [500k]'

---
Создадим функцию для категоризации по количеству детей клиентов

- Нет детей (children == 0)
- Малодетная (children [1, 2])
- Многодетная (Children >= 3)

In [36]:
def children_categ(children):
    if children == 0:
        return 'Нет детей'
    elif children >= 3:
        return 'Многодетная'
    else:
        return 'Малодетная'

---


Создадим функцию для категоризации по Задержкам платежей

- Нет проблем (normal)
- Есть проблемы (debtor)

In [37]:
def debt_categ(debt):
    if debt:
        return 'debtor'
    return 'normal'

---
Функция для создания категорий по леммам

In [38]:
def purpose_lemm_func(purp):
    if purp == 1:
        return 'недвижимость/жилье'
    elif purp == 2:
        return 'автомобиль'
    elif purp == 3:
        return 'образование'
    elif purp == 4:
        return 'свадьба'

---
Запустим функции для создания катеогорий 

In [39]:
scoring_data['purpose_lemm'] = scoring_data['purpose_id'].apply(purpose_lemm_func)

In [40]:
scoring_data['income_categ'] = scoring_data['total_income'].apply(income_categ)

In [41]:
scoring_data['children_categ'] = scoring_data['children'].apply(children_categ)

In [42]:
scoring_data['debt_categ'] = scoring_data['debt'].apply(debt_categ)

---

**Вывод**

Созданы категории по целям. Для дальнейшего использваония в анализе.

Так же создал категории по количеству детей, и уровню заработной платы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для нахождения зависимости построим pivot таблицу взяв за основу данные по клиентам.

In [43]:
data_children = pd.pivot_table(scoring_data, 
               index='children_categ', 
               columns='debt_categ',
               values='debt',
               aggfunc=len)

In [44]:
# Добавление столбца с значением в процентах клиентов задерживающих платеж по категориям.
data_children['stat'] = data_children['debtor'] / (data_children['debtor'] + data_children['normal']) * 100

In [45]:
data_children.sort_values('stat')

debt_categ,debtor,normal,stat
children_categ,,,
Нет детей,1063,13028,7.543822
Многодетная,31,349,8.157895
Малодетная,647,6336,9.265359


**Вывод**

Как видно лучше всех платят клиенты без детей. Так что мы можем уверенно сказать о том что есть связь между наличием детей и возвратом кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [46]:
data_family = pd.pivot_table(scoring_data,
                            index='family_status',
                            columns='debt_categ',
                            values='debt',
                            aggfunc=len)

In [47]:
# Добавление столбца с значением в процентах клиентов задерживающих платеж по категориям.
data_family['stat'] = data_family['debtor'] / (data_family['debtor'] + data_family['normal']) * 100

In [48]:
data_family.sort_values('stat')

debt_categ,debtor,normal,stat
family_status,,,
вдовец / вдова,63,896,6.569343
в разводе,85,1110,7.112971
женат / замужем,931,11408,7.545182
гражданский брак,388,3763,9.347145
Не женат / не замужем,274,2536,9.750890


**Вывод**

Клиенты которые не узаконили свои отношения более склонны к задержкам платежей

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
data_income = pd.pivot_table(scoring_data,
                            index='income_categ',
                            columns='debt_categ',
                            values='debt',
                            aggfunc=len)

In [50]:
data_income['stat'] = data_income['debtor'] / (data_income['debtor'] + data_income['normal']) * 100

In [51]:
data_income.sort_values('stat')

debt_categ,debtor,normal,stat
income_categ,,,
бедность [_60k],49,757,6.079404
Очень богатый [500k],14,208,6.306306
богатый [250k_500K],180,2411,6.947125
высокий средний [170k_250K],383,4522,7.808359
низкий средний [60k_120K],537,5888,8.357977
средний класс [120k_170K],578,5927,8.885473


**Вывод**

Интересный результат в том что люди зарабатывающие до 50к задерживают платежи реже чем люди зарабатывающий до 500.

- Как разные цели кредита влияют на его возврат в срок?

In [52]:
data_purpose = pd.pivot_table(scoring_data,
                            index=['purpose_lemm'],
                            columns='debt_categ',
                            values='debt',
                            aggfunc=len)

In [53]:
data_purpose['stat'] = data_purpose['debtor'] / (data_purpose['debtor'] + data_purpose['normal']) * 100

In [54]:
data_purpose.sort_values('stat')

debt_categ,debtor,normal,stat
purpose_lemm,,,
недвижимость/жилье,782,10029,7.233373
свадьба,186,2138,8.003442
образование,370,3643,9.220035
автомобиль,403,3903,9.359034


**Вывод**

Клиенты берущие кредит для приобретения/проведение операций с жильем, наиболее ответственны и менее склонны нарушать обязательства по выплатам кредита в срок.

## Шаг 4. Общий вывод

In [55]:
data_ender = pd.pivot_table(scoring_data,
                            index=['family_status', 'children_categ'],
                            columns='debt_categ',
                            values='debt',
                            aggfunc=len)

In [56]:
data_ender['stat'] = data_ender['debtor'] / (data_ender['debtor'] + data_ender['normal']) * 100
data_ender.sort_values('stat')

debt_categ                            debtor  normal       stat
family_status         children_categ                           
вдовец / вдова        Нет детей         53.0   794.0   6.257379
женат / замужем       Нет детей        516.0  6952.0   6.909480
в разводе             Нет детей         55.0   729.0   7.015306
женат / замужем       Многодетная       20.0   265.0   7.017544
в разводе             Малодетная        29.0   370.0   7.268170
                      Многодетная        1.0    11.0   8.333333
гражданский брак      Нет детей        229.0  2501.0   8.388278
женат / замужем       Малодетная       395.0  4191.0   8.613171
Не женат / не замужем Нет детей        210.0  2052.0   9.283820
вдовец / вдова        Малодетная        10.0    95.0   9.523810
гражданский брак      Малодетная       151.0  1204.0  11.143911
Не женат / не замужем Малодетная        62.0   476.0  11.524164
гражданский брак      Многодетная        8.0    58.0  12.121212
Не женат / не замужем Многодетная        2.0     8.0  20.000000
вдовец / вдова        Многодетная        NaN     7.0        NaN

Можно смело сказать о том что есть связь между семейным положением, количеством детей и фактом погащения кредита во время.
- Клиента с официально оформленными отношениями (или которые в прошлом были в официальном в браке) и не имеющие детей - самые ответственные
- Клиенты состоящие в неофициальном браке или находящиеся без отношений, при этом имеющие 1 или 2 детей - самые ***не*** ответственные.




## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.